In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, gate_name, input_nodes, hidden_1_nodes, output_nodes, learning_rate):
        
        self.name = gate_name
        
        # 2층 hidden layer unit : 8 개 가정
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_1_nodes)  
        self.b2 = np.random.rand(hidden_1_nodes)
        
        # 3층 output layer unit : 1 개 
        self.W3 = np.random.rand(hidden_1_nodes, output_nodes)
        self.b3 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
            
        print(self.name + " object is created")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    # accuracy method 1
    def accuracy1(self, test_xdata, test_tdata):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):
            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(test_xdata)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
    
    # accuracy method 2
    def accuracy2(self, test_data):
        
        matched_list = []
        not_matched_list = []
        
        input_data = test_data[ :, 0:-1]
        target_data = test_data[ :, -1]
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.xdata = input_data
        self.tdata = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)

In [7]:
and_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
and_tdata = np.array([0, 0, 0, 1])

input_nodes = 2
output_nodes = 1
hidden_1_nodes = 8

lr = 1e-1  # learning rate
epochs = 5000  # epochs, 반복횟수

# AND Gate 객체 생성 및 training
and_obj = LogicGate("AND", input_nodes, hidden_1_nodes, output_nodes, lr)

start_time = datetime.now()

for count in range(epochs):
    
    for index in range(len(and_xdata)):

        and_obj.train(and_xdata[index], and_tdata[index])
    
    if (count % 500 == 0):
        print("count = ", count, "loss value = ", and_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

AND object is created
count =  0 loss value =  0.15624747557552443
count =  500 loss value =  0.0669067126702356
count =  1000 loss value =  0.021023834873546542
count =  1500 loss value =  0.01162105967421738
count =  2000 loss value =  0.00783324020256535
count =  2500 loss value =  0.005835613467981309
count =  3000 loss value =  0.004616458046429484
count =  3500 loss value =  0.0038008322915664525
count =  4000 loss value =  0.003219608910445803
count =  4500 loss value =  0.0027858930646464802

Elapsed Time =>  0:01:10.938200


In [8]:
# AND Gate prediction
print(and_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

(true_list, false_list) = and_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = and_obj.accuracy2(test_data)

AND 

Accuracy =>  1.0
Accuracy =>  1.0


In [9]:
or_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
or_tdata = np.array([0, 1, 1, 1])

input_nodes = 2
output_nodes = 1
hidden_1_nodes = 8

lr = 1e-1  # learning rate
epochs = 5000  # epochs, 반복횟수

# OR Gate 객체 생성 및 training
or_obj = LogicGate("OR", input_nodes, hidden_1_nodes, output_nodes, lr)

start_time = datetime.now()

for count in range(epochs):
    
    for index in range(len(or_xdata)):

        or_obj.train(or_xdata[index], or_tdata[index])
    
    if (count % 500 == 0):
        print("count = ", count, "loss value = ", or_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

OR object is created
count =  0 loss value =  0.05883054779422201
count =  500 loss value =  0.0013525982918947549
count =  1000 loss value =  0.0002623812456191437
count =  1500 loss value =  0.00012223699441183278
count =  2000 loss value =  7.475957642191903e-05
count =  2500 loss value =  5.2105712316984534e-05
count =  3000 loss value =  3.919589162342422e-05
count =  3500 loss value =  3.099181802182593e-05
count =  4000 loss value =  2.53792465009995e-05
count =  4500 loss value =  2.1329770850806003e-05

Elapsed Time =>  0:01:09.793262


In [10]:
# OR Gate prediction
print(or_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

(true_list, false_list) = or_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = or_obj.accuracy2(test_data)

OR 

Accuracy =>  1.0
Accuracy =>  1.0


In [11]:
nand_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
nand_tdata = np.array([1, 1, 1, 0])

input_nodes = 2
output_nodes = 1
hidden_1_nodes = 8

lr = 1e-1  # learning rate
epochs = 5000  # epochs, 반복횟수

# NAND Gate 객체 생성 및 training
nand_obj = LogicGate("NAND", input_nodes, hidden_1_nodes, output_nodes, lr)

start_time = datetime.now()

for count in range(epochs):
    
    for index in range(len(nand_xdata)):

        nand_obj.train(nand_xdata[index], nand_tdata[index])
    
    if (count % 500 == 0):
        print("count = ", count, "loss value = ", nand_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

NAND object is created
count =  0 loss value =  3.364933037362056
count =  500 loss value =  0.08586247374967437
count =  1000 loss value =  0.023555828034499984
count =  1500 loss value =  0.012494900693616187
count =  2000 loss value =  0.00825133266823765
count =  2500 loss value =  0.006069721875095245
count =  3000 loss value =  0.0047595748293821185
count =  3500 loss value =  0.0038929444366448125
count =  4000 loss value =  0.0032806059697391555
count =  4500 loss value =  0.0028267206345111546

Elapsed Time =>  0:01:02.938604


In [12]:
# NAND Gate prediction
print(nand_obj.name, "\n")

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

(true_list, false_list) = nand_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = nand_obj.accuracy2(test_data)

NAND 

Accuracy =>  1.0
Accuracy =>  1.0


In [13]:
xor_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
xor_tdata = np.array([0, 1, 1, 0])

input_nodes = 2
output_nodes = 1
hidden_1_nodes = 8

lr = 1e-1  # learning rate
epochs = 5000  # epochs, 반복횟수

# XOR Gate 객체 생성 및 training
xor_obj = LogicGate("XOR", input_nodes, hidden_1_nodes, output_nodes, lr)

start_time = datetime.now()

for count in range(epochs):
    
    for index in range(len(xor_xdata)):

        xor_obj.train(xor_xdata[index], xor_tdata[index])
    
    if (count % 500 == 0):
        print("count = ", count, "loss value = ", xor_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

XOR object is created
count =  0 loss value =  2.2619159970108025
count =  500 loss value =  0.6496020406095162
count =  1000 loss value =  0.1068764371600119
count =  1500 loss value =  0.03266039656497023
count =  2000 loss value =  0.017201263643066622
count =  2500 loss value =  0.011231957881682877
count =  3000 loss value =  0.008184971179890671
count =  3500 loss value =  0.0063702248339719755
count =  4000 loss value =  0.005178820450355171
count =  4500 loss value =  0.004342498810905633

Elapsed Time =>  0:01:15.522455


In [14]:
# XOR Gate prediction
print(xor_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

(true_list, false_list) = xor_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = xor_obj.accuracy2(test_data)

XOR 

Accuracy =>  1.0
Accuracy =>  1.0
